In [1]:

import matplotlib.pyplot as plt
from keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
import time
from PIL import Image

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import shutil
# Clear any logs from previous runs
shutil.rmtree('logs', ignore_errors=True)

In [4]:

def VGG1():
    model = Sequential()
    model.add(Input(shape=(200, 200, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    # vgg1_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/vgg1_callback_dir" + str(time.time()), update_freq = 'iteration')
    return model

In [5]:

def VGG2():
    model = Sequential()
    model.add(Input(shape=(200, 200, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate=0.005, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
        


In [6]:

def VGG3():
    model = Sequential()
    model.add(Input(shape=(200, 200, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate=0.007, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
def train(model, train_dir, test_dir, max_epochs=20):
    datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    train_it = datagen.flow_from_directory(train_dir, class_mode='binary', batch_size=40, target_size=(200, 200))
    test_it = datagen.flow_from_directory(test_dir, class_mode='binary', batch_size=10, target_size=(200, 200))
    train_steps_per_epoch = len(train_it)
    test_steps_per_epoch = len(test_it)
    print(f"Train steps: {train_steps_per_epoch}, Test steps: {test_steps_per_epoch}")
    history = {'epoch_train_loss': [], 'epoch_train_accuracy': [], 'epoch_val_loss': [], 'epoch_val_accuracy': [],
                'iter_train_loss': [], 'iter_train_accuracy': [], 'iter_val_loss': [], 'iter_val_accuracy': []}
    writer = tf.summary.create_file_writer(f"logs/{model.name}")
    step = 0
    for epoch in range(max_epochs):
        print(f"Epoch {epoch + 1}/{max_epochs}:")
        epoch_loss = []
        epoch_accuracy = []
        epoch_val_loss = []
        epoch_val_accuracy = []

        for batch_index in range(train_steps_per_epoch):
            X_batch, y_batch = next(train_it)
            loss, accuracy = model.train_on_batch(X_batch, y_batch)

            epoch_loss.append(loss)
            epoch_accuracy.append(accuracy)

            history['iter_train_loss'].append(loss)
            history['iter_train_accuracy'].append(accuracy)
            
            loss_val,acc_val=model.evaluate(test_it,steps=test_steps_per_epoch,verbose=0)
            epoch_val_loss.append(loss_val)
            epoch_val_accuracy.append(acc_val)
            history['iter_val_loss'].append(loss_val)
            history['iter_val_accuracy'].append(acc_val)
            with writer.as_default():
                tf.summary.scalar('Testing loss', loss_val, step=step)
                tf.summary.scalar('Testing accuracy', acc_val, step=step)
                tf.summary.scalar('Training loss', loss, step=step)
                tf.summary.scalar('Training accuracy', accuracy, step=step)
                writer.flush()
            step+=1
            
        avg_val_loss = sum(epoch_val_loss) / len(epoch_val_loss)
        avg_val_accuracy = sum(epoch_val_accuracy) / len(epoch_val_accuracy)
        avg_train_loss = sum(epoch_loss) / len(epoch_loss)
        avg_train_accuracy = sum(epoch_accuracy) / len(epoch_accuracy)
        history['epoch_train_loss'].append(avg_train_loss)
        history['epoch_train_accuracy'].append(avg_train_accuracy)
        history['epoch_val_loss'].append(avg_val_loss)
        history['epoch_val_accuracy'].append(avg_val_accuracy)

    return history

In [245]:
# def train(model, train_dir, test_dir):
#     datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20)
#     train_it = datagen.flow_from_directory(train_dir, class_mode='binary', batch_size=40, target_size=(200, 200))
#     test_it = datagen.flow_from_directory(test_dir, class_mode='binary', batch_size=10, target_size=(200, 200))
    
#     train_steps_per_epoch = len(train_it) // train_it.batch_size
#     test_steps_per_epoch = len(test_it) // test_it.batch_size
#     log_dir = "logs/vgg1_callback_dir"
#     vgg1_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')

#     history = model.fit(train_it, steps_per_epoch=train_steps_per_epoch, 
#                         validation_data=test_it, validation_steps=test_steps_per_epoch, 
#                         epochs=20, verbose=2, 
#                         callbacks=[vgg1_callback] )

#     return history

In [246]:
def test(model,test_dir):
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_it = datagen.flow_from_directory(test_dir, class_mode='binary', batch_size=10, target_size=(200, 200))
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=2)
    print(f'Accuracy: {acc*100}')


In [247]:
def train_augmentation(model,train_dir, test_dir):
    datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    train_it = datagen.flow_from_directory(train_dir, class_mode='binary', batch_size=40, target_size=(200, 200))
    test_it = datagen.flow_from_directory(test_dir, class_mode='binary', batch_size=10, target_size=(200, 200))
    train_steps_per_epoch = len(train_it) // train_it.batch_size
    test_steps_per_epoch = len(test_it) // test_it.batch_size
    history = model.fit(train_it, steps_per_epoch=train_steps_per_epoch, validation_data=test_it, validation_steps=test_steps_per_epoch, epochs=20, verbose=2)
    return history

In [9]:
import matplotlib.pyplot as plt
def summarize_diagnostics(history, txt):
    # Plot two subplots one of accuracy and other of loss for iterations
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history['iter_train_loss'], color='blue', label='train')
    plt.plot(history['iter_val_loss'], color='orange', label='test')
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history['iter_train_accuracy'], color='blue', label='train')
    plt.plot(history['iter_val_accuracy'], color='orange', label='test')
    plt.show()
    # Plot two subplots one of accuracy and other of loss for epochs
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history['epoch_train_loss'], color='blue', label='train')
    plt.plot(history['epoch_val_loss'], color='orange', label='test')
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history['epoch_train_accuracy'], color='blue', label='train')
    plt.plot(history['epoch_val_accuracy'], color='orange', label='test')
    plt.show()
    # Save the plot
    plt.tight_layout()
    plt.savefig(txt+ "_plot.png")
    plt.close()
    

In [248]:
# import matplotlib.pyplot as plt

# def summarize_diagnostics(history, txt):
#     fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))

#     # Plot Loss
#     axes[0].set_title('Cross Entropy Loss')
#     axes[0].plot(history.history['loss'], color='blue', label='train')
#     axes[0].plot(history.history['val_loss'], color='orange', label='test')
#     axes[0].legend(['train', 'test'], loc='upper right')
#     axes[0].set_ylabel('Loss')
#     axes[0].set_xlabel('Epoch')

#     # Plot Accuracy
#     axes[1].set_title('Classification Accuracy')
#     axes[1].plot(history.history['accuracy'], color='blue', label='train')
#     axes[1].plot(history.history['val_accuracy'], color='orange', label='test')
#     axes[1].legend(['train', 'test'], loc='upper right')
#     axes[1].set_ylabel('Accuracy')
#     axes[1].set_xlabel('Epoch')

#     plt.tight_layout()
#     plt.savefig(txt + '_plot.png')
#     plt.close() 


In [12]:
import io
from matplotlib import pyplot
from skimage.transform import resize
def plot_to_image(figure):
    buf = io.BytesIO()
    pyplot.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    pyplot.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_grid(predictions, images):
    figure = pyplot.figure(figsize=(10, 10))
    num_images = len(images)
    permutation = np.random.permutation(num_images)
    
    # Shuffle images and predictions based on the permutation
    shuffled_images = [images[i] for i in permutation]
    shuffled_predictions = [predictions[i] for i in permutation]
    for i in range(40):
        # Start next subplot.
        pyplot.subplot(5, 8, i + 1, title= 'Bear' if shuffled_predictions[i] < 0.5 else 'Koala')
        pyplot.xticks([])
        pyplot.yticks([])
        pyplot.grid(False)
        # Resize the image to 200x200 pixels before displaying it
        resized_image = resize(shuffled_images[i], (200, 200))
        pyplot.imshow(resized_image)

    return figure

def load_image(filename):
     # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    # center pixel data
    img = img.astype('float32')
    # img = img - [123.68, 116.779, 103.939]
    return img

In [ ]:
def write_image(model, test_images, images):
    file_writer = tf.summary.create_file_writer(f"logs/{model.name}/test_images")
    predictions = []
    for i in test_images:
        prediction = model.predict(i)            #i has shape (1,200,200,3)
        predictions.append(prediction[0][0])

    figure = image_grid(predictions, images)

    with file_writer.as_default():
        tf.summary.image("Test Image", plot_to_image(figure), step=0)

In [13]:
# Create test_images
test_images = []
images = []
# load test_images from dataset_koala_vs_bear/test/koala
for file in os.listdir('dataset_koala_vs_bear/test/koala/'):
    img= load_image('dataset_koala_vs_bear/test/koala/' + file)
    images.append(Image.open('dataset_koala_vs_bear/test/koala/' + file))
    test_images.append(img)


for file in os.listdir('dataset_koala_vs_bear/test/bear/'):
    img= load_image('dataset_koala_vs_bear/test/bear/' + file)
    images.append(Image.open('dataset_koala_vs_bear/test/bear/' + file))
    test_images.append(img)

In [14]:
print(len(test_images))
print(len(images))

40
40


In [16]:
# Creating the table
# Model | Train Accuracy | Test Accuracy | Train Time | Test Time
train_accuracy = []
train_loss = []
test_accuracy = []
train_time = []
num_params = []

## VGG 1

In [15]:
vgg1 = VGG1()
start_time_vgg1 = time.time()
history = train(vgg1,'dataset_koala_vs_bear/train/', 'dataset_koala_vs_bear/test/')
end_time_vgg1 = time.time()
train_time_vgg1 = end_time_vgg1 - start_time_vgg1
train_time.append(train_time_vgg1)
print(f"Time taken for VGG1: {train_time_vgg1}")

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Train steps: 4, Test steps: 4
Epoch 1/20:
Epoch 2/20:


KeyboardInterrupt: 

In [253]:
write_image(vgg1, test_images, images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [ ]:
# model.evaluate(train , steps=len(test_images), verbose=2)

In [254]:
train_loss, train_accuracy = history.history['loss'], history.history['accuracy']
test_loss, test_accuracy = history.history['val_loss'], history.history['val_accuracy']
# Add scalers to tensorboard like training loss, training accuracy, testing accuracy
# Add graphs to tensorboard like training loss vs epochs, training accuracy vs epochs, testing accuracy vs epochs
# tf.summary.scalar('Training Loss', train_loss, step=0)

num_params = vgg1.count_params()

print(f"Number of parameters: {num_params}")
print(f"Training Loss: {len(train_loss)}")
summarize_diagnostics(history, 'vgg1')

Number of parameters: 40961153
Training Loss: 20


In [255]:
print(len(test_loss))

20


In [256]:
start_time_vgg1_test = time.time()
test(vgg1,'dataset_koala_vs_bear/test/')
end_time_vgg1_test = time.time()
time_vgg1_test = end_time_vgg1_test - start_time_vgg1_test
print(f"Time taken for training VGG1: {time_vgg1_train}")
print(f"Time taken for testing VGG1: {time_vgg1_test}")
summarize_diagnostics(history,'vgg1')

Found 40 images belonging to 2 classes.
4/4 - 1s - 155ms/step - accuracy: 0.6500 - loss: 0.5403
Accuracy: 64.99999761581421
Time taken for training VGG1: 93.48743796348572
Time taken for testing VGG1: 0.8952891826629639


In [28]:
!tensorboard --logdir logs

^C


In [15]:
vgg2 = VGG2()
start_time_vgg2 = time.time()
history = train(vgg2,'dataset_koala_vs_bear/train/', 'dataset_koala_vs_bear/test/')
end_time_vgg2 = time.time()
time_vgg2 = end_time_vgg2 - start_time_vgg2
print(f"Time taken for VGG2: {time_vgg2}")

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/20
4/4 - 7s - 2s/step - accuracy: 0.4750 - loss: 4.5228 - val_accuracy: 0.5000 - val_loss: 0.7120
Epoch 2/20
4/4 - 6s - 2s/step - accuracy: 0.5000 - loss: 0.7117 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/20
4/4 - 6s - 2s/step - accuracy: 0.5562 - loss: 0.6841 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 4/20
4/4 - 7s - 2s/step - accuracy: 0.5437 - loss: 0.6788 - val_accuracy: 0.5500 - val_loss: 0.6776
Epoch 5/20
4/4 - 7s - 2s/step - accuracy: 0.5750 - loss: 0.6597 - val_accuracy: 0.5500 - val_loss: 0.6626
Epoch 6/20
4/4 - 7s - 2s/step - accuracy: 0.7063 - loss: 0.6295 - val_accuracy: 0.5750 - val_loss: 0.6518
Epoch 7/20
4/4 - 7s - 2s/step - accuracy: 0.6938 - loss: 0.6009 - val_accuracy: 0.6250 - val_loss: 0.6159
Epoch 8/20
4/4 - 7s - 2s/step - accuracy: 0.6687 - loss: 0.5760 - val_accuracy: 0.7000 - val_loss: 0.5968
Epoch 9/20
4/4 - 7s - 2s/step - accuracy: 0.8000 - loss: 0.5164 - val_a

In [16]:
start_time_vgg2_test = time.time()
test(vgg2,'dataset_koala_vs_bear/test/')
end_time_vgg2_test = time.time()
summarize_diagnostics(history,'vgg2')
time_vgg2_test = end_time_vgg2_test - start_time_vgg2_test
print(f"Time taken for testing VGG2: {time_vgg2_test}")

Found 40 images belonging to 2 classes.


4/4 - 1s - 281ms/step - accuracy: 0.8000 - loss: 0.6288
Accuracy: 80.0000011920929
Time taken for testing VGG2: 1.5418579578399658


In [8]:
vgg3 = VGG3()
start_time_vgg3 = time.time()
history = train(vgg3,'dataset_koala_vs_bear/train/', 'dataset_koala_vs_bear/test/')
end_time_vgg3 = time.time()
time_vgg3 = end_time_vgg3 - start_time_vgg3
print(f"Time taken for VGG3: {time_vgg3}")

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/20
4/4 - 9s - 2s/step - accuracy: 0.4812 - loss: 4.2923 - val_accuracy: 0.5250 - val_loss: 0.7028
Epoch 2/20
4/4 - 7s - 2s/step - accuracy: 0.4938 - loss: 0.6966 - val_accuracy: 0.6000 - val_loss: 0.6911
Epoch 3/20
4/4 - 8s - 2s/step - accuracy: 0.5375 - loss: 0.7138 - val_accuracy: 0.4750 - val_loss: 0.7205
Epoch 4/20
4/4 - 8s - 2s/step - accuracy: 0.4938 - loss: 0.7135 - val_accuracy: 0.6750 - val_loss: 0.6747
Epoch 5/20
4/4 - 8s - 2s/step - accuracy: 0.5813 - loss: 0.6762 - val_accuracy: 0.5000 - val_loss: 0.6831
Epoch 6/20
4/4 - 7s - 2s/step - accuracy: 0.5750 - loss: 0.6675 - val_accuracy: 0.6250 - val_loss: 0.6710
Epoch 7/20
4/4 - 7s - 2s/step - accuracy: 0.6562 - loss: 0.6567 - val_accuracy: 0.6750 - val_loss: 0.6522
Epoch 8/20
4/4 - 7s - 2s/step - accuracy: 0.6750 - loss: 0.6344 - val_accuracy: 0.7250 - val_loss: 0.6301
Epoch 9/20
4/4 - 7s - 2s/step - accuracy: 0.6375 - loss: 0.6198 - val_a

In [9]:
start_time_vgg3_test = time.time()
test(vgg3,'dataset_koala_vs_bear/test/')
end_time_vgg3_test = time.time()
summarize_diagnostics(history,'vgg3')
time_vgg3_test = end_time_vgg3_test - start_time_vgg3_test
print(f"Time taken for testing VGG3: {time_vgg3_test}")

Found 40 images belonging to 2 classes.


4/4 - 1s - 255ms/step - accuracy: 0.7000 - loss: 0.5962
Accuracy: 69.9999988079071
Time taken for testing VGG3: 1.4439589977264404


In [8]:
vgg3_aug = VGG3()
start_time_vgg3_aug = time.time()
history = train_augmentation(vgg3_aug,'dataset_koala_vs_bear/train/', 'dataset_koala_vs_bear/test/')
end_time_vgg3_aug = time.time()
time_vgg3_aug = end_time_vgg3_aug - start_time_vgg3_aug
print(f"Time taken for VGG3 with augmentation: {time_vgg3_aug}")

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/20
4/4 - 9s - 2s/step - accuracy: 0.5000 - loss: 3.1542 - val_accuracy: 0.5000 - val_loss: 0.8025
Epoch 2/20
4/4 - 8s - 2s/step - accuracy: 0.5312 - loss: 0.7539 - val_accuracy: 0.5000 - val_loss: 0.6872
Epoch 3/20
4/4 - 8s - 2s/step - accuracy: 0.5125 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6785
Epoch 4/20
4/4 - 8s - 2s/step - accuracy: 0.5625 - loss: 0.6766 - val_accuracy: 0.5000 - val_loss: 0.6814
Epoch 5/20
4/4 - 7s - 2s/step - accuracy: 0.6062 - loss: 0.6650 - val_accuracy: 0.7000 - val_loss: 0.6670
Epoch 6/20
4/4 - 8s - 2s/step - accuracy: 0.7250 - loss: 0.6445 - val_accuracy: 0.7250 - val_loss: 0.6548
Epoch 7/20
4/4 - 7s - 2s/step - accuracy: 0.7250 - loss: 0.6321 - val_accuracy: 0.7000 - val_loss: 0.6389
Epoch 8/20
4/4 - 7s - 2s/step - accuracy: 0.6250 - loss: 0.6282 - val_accuracy: 0.6500 - val_loss: 0.6255
Epoch 9/20
4/4 - 7s - 2s/step - accuracy: 0.6687 - loss: 0.5933 - val_a

In [9]:
start_time_vgg3_aug_test = time.time()
test(vgg3_aug,'dataset_koala_vs_bear/test/')
end_time_vgg3_aug_test = time.time()
summarize_diagnostics(history,'vgg3_aug')
print(f"Time taken for testing VGG3 with augmentation: {end_time_vgg3_aug_test - start_time_vgg3_aug_test}")

Found 40 images belonging to 2 classes.


4/4 - 1s - 244ms/step - accuracy: 0.7500 - loss: 0.4567
Accuracy: 75.0
Time taken for testing VGG3 with augmentation: 1.5705618858337402
